In [1]:
# Load packages and settings
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 50)
import seaborn as sns


import matplotlib.pyplot as plt
%matplotlib widget
plt.rcParams['figure.figsize'] = (12,8)
plt.rcParams["image.cmap"] = "tab10"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)
fs_label = 16
parameters = {
                'axes.labelsize': fs_label,
                'axes.titlesize': fs_label+4,
                'xtick.labelsize': fs_label,
                'ytick.labelsize': fs_label, 
                'legend.fontsize': fs_label, 
                'lines.markersize': 10,
                'lines.linewidth': 3
             }
plt.rcParams.update(parameters)
%matplotlib widget
from matplotlib import cm # Colormaps
import matplotlib.colors as colors
# cmap = plt.cm.get_cmap('Dark2',len(ageGroups))

import locale
import matplotlib.dates as mdates
locale.setlocale(locale.LC_ALL, '') 
locale.setlocale(locale.LC_TIME,"Danish")
# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))
# ax1.spines['top'].set_visible(False) 

import os
# import csv
import math

import datetime
from datetime import date


# ax.legend(bbox_to_anchor=(1.04,0.5), loc="center left")

saveFigures = True
# saveFigures = False
print('saveFigures is set to: '+str(saveFigures))

print('Done loading packages')

# Define running mean functions
def rnMean(data,meanWidth):
    return np.convolve(data, np.ones(meanWidth)/meanWidth, mode='valid')
def rnTime(t,meanWidth):
    return t[math.floor(meanWidth/2):-math.ceil(meanWidth/2)+1]

saveFigures is set to: True
Done loading packages


In [7]:
# Define paths
rootdir_data = os.getcwd() +"\\..\\DanskeData\\" 

path_data = rootdir_data + "ssi_data\\"
path_dash = rootdir_data + "ssi_dashboard\\"
path_vacc = rootdir_data + "ssi_vacc\\"
path_figs = os.getcwd() +"\\..\\Figures\\" 

# path_figs = path_figs + "Immunity_SSI\\"

In [15]:
latestsubdir = list(os.walk(path_dash))[0][1][-1]
latestdir = path_dash + latestsubdir
df = pd.read_csv(latestdir+'/Regionalt_DB/18_fnkt_alder_uge_testede_positive_nyindlagte.csv',encoding='latin1',delimiter = ';')

df['Nyindlagte pr. 100.000 borgere'] = pd.to_numeric(df['Nyindlagte pr. 100.000 borgere'].str.replace(',','.'))

# df_07['Dato'] = pd.to_datetime(df_07['Dato'])
df.tail()

,Uge,Aldersgruppe,Testede pr. 100.000 borgere,Positive pr. 100.000 borgere,Nyindlagte pr. 100.000 borgere,Antal testede,Antal positive
926,2022-W05,16-19,16112,6224,24.4,44171,17063
927,2022-W05,20-39,19708,5266,36.5,293318,78379
928,2022-W05,40-64,23119,4640,28.6,436752,87649
929,2022-W05,65-79,10226,1887,45.9,92423,17051
930,2022-W05,80+,11687,1220,118.8,34123,3563


In [23]:
weekShort = df.Uge.apply(lambda x: x[-2:])
# dfDate
df['Date'] = df.Uge.apply(lambda x: datetime.datetime.strptime(x + '-1', "%Y-W%W-%w"))

In [27]:
curdf = df[df.Aldersgruppe == '00-02']
curdf.tail()

,Uge,Aldersgruppe,Testede pr. 100.000 borgere,Positive pr. 100.000 borgere,Nyindlagte pr. 100.000 borgere,Antal testede,Antal positive,Date
886,2022-W01,00-02,6834,1369,37.6,12738,2551,2022-01-03
895,2022-W02,00-02,12228,2591,56.9,22790,4829,2022-01-10
904,2022-W03,00-02,14563,3463,84.2,27143,6455,2022-01-17
913,2022-W04,00-02,11454,3208,79.4,21349,5979,2022-01-24
922,2022-W05,00-02,9508,3258,96.0,17726,6073,2022-01-31


In [48]:
# curAdmCount

In [98]:
curAdm = curdf['Nyindlagte pr. 100.000 borgere']
curCase = curdf['Antal positive']
curInci = curdf['Positive pr. 100.000 borgere']
curPopCount = np.mean((100000*curCase/curInci)[-5:])
curPopCount 

curAdmCount = curPopCount*curAdm/100000

fig,allAxes = plt.subplots(2,1,sharex=True,figsize=(13,12))
# ax1,ax2,ax3 = allAxes.flatten()
ax1,ax2 = allAxes.flatten()

ax1.plot(curdf.Date,curAdm,'k.-')
ax1_2 = ax1.twinx()
ax1_2.plot(curdf.Date,curAdmCount,'k.-')

# ax1.set_ylabel('Nyindlæggelser per 100.000')
# ax1_2.set_ylabel('Antal nyindlæggelser')
ax1.set_ylabel('New admissions per 100.000')
ax1_2.set_ylabel('New admissions')

ax1.set_ylim(bottom=0)
ax1_2.set_ylim(bottom=0)

ax2.plot(curdf.Date,curdf['Positive pr. 100.000 borgere'],'k.-')
ax2_2 = ax2.twinx()
ax2_2.plot(curdf.Date,curdf['Antal positive'],'k.-')

ax2.set_ylabel('Cases per 100.000')
ax2_2.set_ylabel('Cases')
ax2.set_ylim(bottom=0)
ax2_2.set_ylim(bottom=0)

ax1.grid(axis='y')
ax2.grid(axis='y')

ax1.set_title('0 to 2 year olds')

if saveFigures:
    fig.savefig(path_figs+'CasesAndHospitalizations_Full_0-2')

ax1.set_xlim(left=np.datetime64('2021-09-01'))


if saveFigures:
    fig.savefig(path_figs+'CasesAndHospitalizations_0-2')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [93]:
# print(curAdmCount[:-1].values/curCase[1:].values)
# print(curAdmCount.values/curCase.values)
print(curCase.values[-5:])
print(curAdmCount.values[-5:])
print(curCase[2:].values[-5:])
print(curAdmCount[:-2].values[-5:])

[2551 4829 6455 5979 6073]
[ 70.0785712  106.04975269 156.93126848 147.9850679  178.92401156]
[2551 4829 6455 5979 6073]
[ 40.07152342  79.21115095  70.0785712  106.04975269 156.93126848]


In [101]:
caseHosp = curAdmCount/curCase
caseHospm1 = curAdmCount[1:].values/curCase[:-1].values
caseHospm2 = curAdmCount[2:].values/curCase[:-2].values

fig,ax1 = plt.subplots()
ax1.plot(curdf.Date,caseHosp,'k.:',markersize=2,linewidth=0.5,label='Admission divided by cases (same week)')
ax1.plot(rnTime(curdf.Date,7),rnMean(caseHosp,7),'k',label='7-day running mean')
ax1.plot(curdf.Date[:-1],caseHospm1,'r.:',markersize=2,linewidth=0.5,label='Admission divided by cases (week earlier)')
ax1.plot(rnTime(curdf.Date[:-1],7),rnMean(caseHospm1,7),'r',label='7-day running mean')
ax1.plot(curdf.Date[:-2],caseHospm2,'b.:',markersize=2,linewidth=0.5,label='Admission divided by cases (two weeks earlier)')
ax1.plot(rnTime(curdf.Date[:-2],7),rnMean(caseHospm2,7),'b',label='7-day running mean')

ax1.set_ylim(bottom=0)
ax1.set_ylim(top=0.4)

ax1.legend(loc='upper right',fontsize=13)
ax1.grid()

ax1.set_ylabel('Case hospitalization rate')
ax1.set_title('0 to 2 year olds')

if saveFigures:
    fig.savefig(path_figs+'CaseHospitalization_Full_0-2')

ax1.set_ylim(top=0.14)
ax1.set_xlim(left=np.datetime64('2021-09-01'))
if saveFigures:
    fig.savefig(path_figs+'CaseHospitalization_0-2')

<ipython-input-101-a7fa0f04c618>:2: RuntimeWarning: divide by zero encountered in true_divide
  caseHospm1 = curAdmCount[1:].values/curCase[:-1].values
<ipython-input-101-a7fa0f04c618>:2: RuntimeWarning: invalid value encountered in true_divide
  caseHospm1 = curAdmCount[1:].values/curCase[:-1].values
<ipython-input-101-a7fa0f04c618>:3: RuntimeWarning: divide by zero encountered in true_divide
  caseHospm2 = curAdmCount[2:].values/curCase[:-2].values
<ipython-input-101-a7fa0f04c618>:3: RuntimeWarning: invalid value encountered in true_divide
  caseHospm2 = curAdmCount[2:].values/curCase[:-2].values


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
# # Collect age-specific cases for all municipalities
# allSubDirs = list(os.walk(path_dash))[0][1]

# df = pd.DataFrame()

# for curSubDir in allSubDirs:
#     curdir = path_dash + curSubDir
#     curfilepath = curdir+'/Regionalt_DB/21_noegletal_pr_region_pr_fnkt_alder.csv'

#     # Check if file was included at the time. The "Kommune/17" file wasn't included until 2021-09-22 
#     if os.path.isfile(curfilepath):
#         curdf = pd.read_csv(curfilepath,encoding='latin1',delimiter = ';')
        
#         df = pd.concat([df,curdf])

# df
# # # Set dtypes
# # df.Kommune = df.Kommune.fillna(0)  # All NaN kommuner is set to zero
# # df['Kommune'] = df['Kommune'].astype(int)
# # df['Dagsdato'] = pd.to_datetime(df['Dagsdato'])
# # df['Bekræftede tilfælde'] = pd.to_numeric(df['Bekræftede tilfælde'])
# # df['Aldersgruppe'] = df.Aldersgruppe.replace('00-02','0-2')
# # df['Aldersgruppe'] = df.Aldersgruppe.replace('03-05','3-5')
# # df['Aldersgruppe'] = df.Aldersgruppe.replace('06-11','6-11')

# # # df['Forskel'] = df['Bekræftede tilfælde'].diff().fillna(0).astype(int,errors='ignore')

,Region,Aldersgruppe,Bekræftede tilfælde,Indlagte på intensiv afdeling,Indlagte,Dato
0,NaN,0-2,30,0,5,22/09/2021
1,Hovedstaden,0-2,2608,0,136,22/09/2021
2,Midtjylland,0-2,943,0,27,22/09/2021
3,Nordjylland,0-2,375,0,11,22/09/2021
4,Sjælland,0-2,537,0,33,22/09/2021
5,Syddanmark,0-2,828,0,21,22/09/2021
6,NaN,3-5,22,0,0,22/09/2021
7,Hovedstaden,3-5,3700,0,14,22/09/2021
8,Midtjylland,3-5,1382,0,7,22/09/2021
9,Nordjylland,3-5,503,0,3,22/09/2021
